# Übung 3.1: Web-Scraping

Die US Social Security Administration erstellt für jedes Jahr eine [Statistik der häufigsten Babynamen](https://www.ssa.gov/OACT/babynames/) (in Deutschland gibt es diese Statistik leider nur kommerziell für 10 € (!) von der [Gesellschaft für Deutsche Sprache](https://gfds.de/vornamen/beliebteste-vornamen/) – soviel zum Thema [Informationsfreiheitsgesetz](https://de.wikipedia.org/wiki/Informationsfreiheitsgesetz)).

Die vollständige Liste erhält man über eine POST Request an die URL `https://www.ssa.gov/cgi-bin/popularnames.cgi` mit den Parametern
- year (1880 und später)
- top (bis 1000)


## Aufgabe

1. Erstellen Sie eine Funktion, die für ein gegebenes Jahr eine Liste der 1000 beliebtesten amerikanischen Mädchen- und Jungennamen mit der jeweiligen Häufigkeit des Namens zurückgibt. Die Datenstruktur könnte z.B. so aussehen:
``` python
{
  'male':  ( {'John': 10000, 'Michael': 9000 }, [ 'John', 'Michael' ] ),
  'female':  ( {'Mary': 10000, 'Linda': 9000 }, [ 'Mary', 'Linda' ] )
}
```
2. Bestimmen Sie für die Jahre **1900**, **1950**, **2000** und **2018**
  - Die jeweils 10 beliebtesten Mädchen- und Jungennamen,
  - den häufigsten Namen, den sowohl Mädchen als auch Jungen haben.


Das (korrigierte) Beispiel zum Thema Web-Scraping aus der Vorlesung finden Sie auf [GitLab](https://gitlab.com/fh-swf/fb-in/skriptsprachen/notebooks/blob/master/lecture_03/03_http_request_and_web_scraping.ipynb) (auf [GitHub](https://github.com/fhswf/skriptsprachen/blob/master/lecture_03/03_http_request_and_web_scraping.ipynb) scheint aktuell die Anzeige von Jupyter Notebooks nicht zu funktionieren). 

In [17]:
import requests
import lxml.html as lh

url = 'https://www.ssa.gov/cgi-bin/popularnames.cgi'


def getNames(year):
    resp = requests.post(url, { 'year': year, 'top': 1000, 'number': 'n'})
    doc = lh.fromstring(resp.text)
    
    # Wähle alle Tabellenzeilen (`<tr>`) via XPath aus
    tr_elements = doc.xpath('//table[2]//td[2]//tr')

    male   = (dict(), [])
    female = (dict(), [])
    
    for row in tr_elements[1:]:
        if len(row) == 5:
            maleName = row[1].text_content()
            maleCount = int(row[2].text_content().replace(',', ''))
            femaleName = row[3].text_content()
            femaleCount = int(row[4].text_content().replace(',', ''))
            male[0][maleName] = maleCount
            male[1].append(maleName)
            female[0][femaleName] = femaleCount
            female[1].append(femaleName)
            
    return { 'male': male, 'female': female }



In [15]:
year = 2018
res = getNames(year)
res
    

{'male': ({'Liam': 19837,
   'Noah': 18267,
   'William': 14516,
   'James': 13525,
   'Oliver': 13389,
   'Benjamin': 13381,
   'Elijah': 12886,
   'Lucas': 12585,
   'Mason': 12435,
   'Logan': 12352,
   'Alexander': 11989,
   'Ethan': 11854,
   'Jacob': 11770,
   'Michael': 11620,
   'Daniel': 11173,
   'Henry': 10649,
   'Jackson': 10323,
   'Sebastian': 10054,
   'Aiden': 9979,
   'Matthew': 9924,
   'Samuel': 9734,
   'David': 9697,
   'Joseph': 9555,
   'Carter': 9312,
   'Owen': 9288,
   'Wyatt': 9127,
   'John': 9119,
   'Jack': 8834,
   'Luke': 8577,
   'Jayden': 8553,
   'Dylan': 8549,
   'Grayson': 8538,
   'Levi': 8440,
   'Isaac': 8417,
   'Gabriel': 8335,
   'Julian': 8307,
   'Mateo': 8169,
   'Anthony': 8003,
   'Jaxon': 7701,
   'Lincoln': 7368,
   'Joshua': 7364,
   'Christopher': 7261,
   'Andrew': 7234,
   'Theodore': 7020,
   'Caleb': 6929,
   'Ryan': 6905,
   'Asher': 6802,
   'Nathan': 6790,
   'Thomas': 6779,
   'Leo': 6719,
   'Isaiah': 6614,
   'Charles': 660

In [16]:
for year in (1900, 1950, 2000, 2018):
    res = getNames(year)

    print(f"Top ten in {year}:")
    for i in range(10):
        print(f"{i+1:2d} {res['female'][1][i]:15s}  {res['male'][1][i]:15s}")
    
    male = res['male'][0]
    female = res['female'][0]
    
    ranking = sorted([ (name, min(male[name], female[name])) for name in male.keys() & female.keys()],\
                     key = lambda pair : pair[1], reverse = True)
    (best, max) = ranking[0]
              
    print()
    print(f'Most popular name given to boys and girls in {year}: {best}  ({max:d})')
    print()

Top ten in 1900:
 1 Mary             John           
 2 Helen            William        
 3 Anna             James          
 4 Margaret         George         
 5 Ruth             Charles        
 6 Elizabeth        Robert         
 7 Florence         Joseph         
 8 Ethel            Frank          
 9 Marie            Edward         
10 Lillian          Henry          

Most popular name given to boys and girls in 1900: Willie  (1351)

Top ten in 1950:
 1 Linda            James          
 2 Mary             Robert         
 3 Patricia         John           
 4 Barbara          Michael        
 5 Susan            David          
 6 Nancy            William        
 7 Deborah          Richard        
 8 Sandra           Thomas         
 9 Carol            Charles        
10 Kathleen         Gary           

Most popular name given to boys and girls in 1950: Terry  (2527)

Top ten in 2000:
 1 Emily            Jacob          
 2 Hannah           Michael        
 3 Madison          Ma